TRABALHO BD

A partir de webscrapping, ir buscar o número de ids escolhido (máx 20) de termos à escolha do utilizador e importar as informações para a base de dados

WEBSCRAPPING

In [174]:
query = input("Defina o termo que pretende pesquisar [bacteria/yeast/human...] \n Se o termo tiver mais do que uma palavra, separe por _: ")
def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(query)
    url_list.append(url)
    return url_list
    
url_get(0)

['https://www.ncbi.nlm.nih.gov/gene/?term=cat']

In [175]:
import requests, sys, json
content = []
for url in url_get(0):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [176]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [177]:
import re
existe = re.findall(r'\s*ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'\s*ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['9685',
 '847',
 '40048',
 '101093891',
 '531682',
 '24248',
 '12359',
 '30068',
 '403474',
 '397568',
 '423600',
 '717406',
 '451116',
 '100388372',
 '100060647',
 '123601576',
 '122483692',
 '100584637',
 '100473949',
 '100171588',
 '117877014']

In [178]:
number_of_genes = int(input("Número de genes a obter (máx: 20): "))
print(number_of_genes)
genes = id_gene[0:number_of_genes]
print(genes)

9
['9685', '847', '40048', '101093891', '531682', '24248', '12359', '30068', '403474']


In [7]:
#BUSCAR LINKS DOS GENES

In [179]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/9685',
 'https://www.ncbi.nlm.nih.gov/nuccore/847',
 'https://www.ncbi.nlm.nih.gov/nuccore/40048',
 'https://www.ncbi.nlm.nih.gov/nuccore/101093891',
 'https://www.ncbi.nlm.nih.gov/nuccore/531682',
 'https://www.ncbi.nlm.nih.gov/nuccore/24248',
 'https://www.ncbi.nlm.nih.gov/nuccore/12359',
 'https://www.ncbi.nlm.nih.gov/nuccore/30068',
 'https://www.ncbi.nlm.nih.gov/nuccore/403474']

In [180]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [181]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
#r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
locus_list = []
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')

	if "Error" not in r2.content.decode('utf-8'):
		locus_list.append(r2.content.decode('utf-8'))
print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       X57223                  2098 bp    DNA     linear   INV 28-FEB-1992
DEFINITION  Meloidogyne arenaria 28S and 5S ribosomal DNA.
ACCESSION   X57223
VERSION     X57223.1
KEYWORDS    28S ribosomal RNA; 5S ribosomal RNA; ribosomal RNA.
SOURCE      Meloidogyne arenaria
  ORGANISM  Meloidogyne arenaria
            Eukaryota; Metazoa; Ecdysozoa; Nematoda; Chromadorea; Rhabditida;
            Tylenchina; Tylenchomorpha; Tylenchoidea; Meloidogynidae;
            Meloidogyninae; Meloidogyne; Meloidogyne incognita group.
REFERENCE   1
  AUTHORS   Vahidi,H., Purac,A., Leblanc,J.M. and Honda,B.M.
  JOURNAL   Unpublished
REFERENCE   2
  AUTHORS   Vahidi,H.
  TITLE     Direct Submission
  JOURNAL   Submitted (11-JAN-1991) Vahidi H., Simon Fraser University,
            Burnaby, Britisch Columbia, Canada, V5A 1S6
REFERENCE   3  (bases 1 to 2098)
  AUTHORS   Vahidi,H.
  TITLE     Direct Submission
  JOURNAL   Submitted (14-JAN-1991) H. Vahidi, SIMON FRASER UNIVERSITY, BURNABY
            BRI

LINK NCBI

In [182]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista


['https://www.ncbi.nlm.nih.gov/nuccore/X57223',
 'https://www.ncbi.nlm.nih.gov/nuccore/X53592',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAAEJ0019801',
 'https://www.ncbi.nlm.nih.gov/nuccore/Z33199']

ACCESSION

In [183]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['X57223 ', 'X53592 ', 'AAAEJ0019801 ', 'Z33199 ']

VERSION

In [184]:
version_list = []

existe = re.findall(r'VERSION.*?(?=KEYWORDS|DBLINK)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for vers in existe:
        m = re.match( r'VERSION\s+(.+)', vers, re.DOTALL )
        version_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
version_list

['X57223.1 ', 'X53592.1 ', 'AAAEJ0019801.1 ', 'Z33199.1 ']

DEFINITION

In [185]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['Meloidogyne arenaria 28S and 5S ribosomal DNA. ',
 'Dog mRNA for glycoprotein gp25L. ',
 'Mus musculus strain C57BL/6J. ',
 'M.capricolum DNA for CONTIG MC272. ']

LOCUS

In [186]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['X57223 2098 bp DNA linear INV 28-FEB-1992 ',
 'X53592 769 bp mRNA linear MAM 30-NOV-1992 ',
 'AAAEJ0019801 21 bp RNA linear ROD 25-MAY-2006 ',
 'Z33199 760 bp DNA linear BCT 18-AUG-1994 ']

LOCUS NAME

In [187]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list


['X57223', 'X53592', 'AAAEJ0019801', 'Z33199']

NUMBER OF BP

In [188]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['2098 bp', '769 bp', '21 bp', '760 bp']

MOLECULE TYPE

In [189]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['DNA linear', 'mRNA linear', 'RNA linear', 'DNA linear']

GENBANK DIVISION

In [190]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['INV', 'MAM', 'ROD', 'BCT']

MODIFICATION DATE

In [191]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['28-FEB-1992', '30-NOV-1992', '25-MAY-2006', '18-AUG-1994']

SOURCE

In [192]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Meloidogyne arenaria ',
 'Canis lupus familiaris (dog) ',
 'Mus musculus (house mouse) ',
 'Mycoplasma capricolum subsp. capricolum ATCC 27343 ']

ORGANISM

In [193]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Meloidogyne arenaria',
 'Canis lupus familiaris',
 'Mus musculus',
 'Mycoplasma capricolum subsp. capricolum ATCC 27343']

FEATURES

In [143]:
misc = []
existe = re.findall(r'FEATURES\s+.*?(?=ORIGIN)', locus, re.DOTALL)
#DOTALL ignora as linhas, pois as features ocupam várias linhas
if existe:
    for feat in existe:
        m = re.match( r'FEATURES\s+(.+)',feat, re.DOTALL )
        misc.append(re.sub(r'/',  ' ', m.group(1) ) )
    for c in misc:
        print(c, "\n")

Location Qualifiers
     source          1..2098
                      organism="Meloidogyne arenaria"
                      mol_type="genomic DNA"
                      db_xref="taxon:6304"
     rRNA            1..139
                      product="28S ribosomal RNA"
     repeat_region   200..2066
                      note="14x 129 bp repeats"
     rRNA            2071..2098
                      product="5S ribosomal RNA"
                      note="duplication"
 

Location Qualifiers
     source          1..769
                      organism="Canis lupus familiaris"
                      mol_type="mRNA"
                      sub_species="familiaris"
                      db_xref="taxon:9615"
                      tissue_lib="pancreas"
     gene            1..769
                      gene="gp25L"
     mRNA            1..769
                      gene="gp25L"
     CDS             6..653
                      gene="gp25L"
                      codon_start=1
                      prod

FEATURES - SOURCE

In [255]:

existe = re.findall(r'source\s+[1-9\.]*', locus, re.DOTALL)
if existe:
    for fsource in existe:
        print( re.sub(r'source\s+', ' ', fsource) )


 1..178
 1..8
 1..8
 1..5
 1..778
 1..8
 1..51
 1..27696
 1..67145
 1..816


FEATURES - EXONS

In [ ]:
existe = re.findall(r'exon\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fexons in existe:
        print( re.sub(r'exon\s+', ' ', fexons) )

FEATURES - INTRON

In [ ]:
existe = re.findall(r'intron\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fintrons in existe:
        print( re.sub(r'intron\s+', ' ', fintrons) )

CDS - JOIN

In [ ]:
existe = re.findall(r'CDS\s+?(join)[1-9\.\,\(\)]*', locus, re.DOTALL)
#(join) procura a palavra join 1x
#procura nrs de 1-9, ponto, virgula e parentises zero ou mais vezes
print(existe[0])
if existe:
    for cds in existe:
        print( re.sub(r'CDS\s+', ' ', cds) )

SEQUENCE/ORIGIN

In [194]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['aagcttagcactctcgtgcatgtgtaacgtcggggcgcaaaatcattgtatacgacttgaatactggtcagggggttgtaatcatagagcagctcttcgctgcgatctgttgagacttagcccttgatcggatgattgtcttttttatatactatagacctgtttatccaagtgtcgtagtatccaagtgtccaagtgtcgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttttggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttttggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttttggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttatggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttttggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagtgcaattgaccaagtgtcgtagggttcaggcaatagggttcccccaggcaatcaaatttcaccaggcatttttggtttagagtctgtgacgatccgattaaaaattgcctcaagcaaagtagaccaaagt

In [195]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[628, 437, 553, 480], [253, 144, 217, 155], [7, 2, 1, 11], [298, 68, 321, 73]]

In [196]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[2098, 769, 21, 760]

JOURNAL

In [259]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES|COMMENT)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994) MEDLINE 94224807 ',
 'Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany, University of Melbourne, Parkville, VIC 3052, Australia ',
 'EMBO J. 7 (6), 1675-1682 (1988) MEDLINE 89005055 ',
 'Unpublished ',
 'Submitted (02-OCT-2002) Molecular Biology, PROIMI, Av. Belgrano y Pje. Caseros, S.M. de Tucuman, Tucuman 4000, Argentina ',
 'Unpublished (2002) ',
 'Unpublished ',
 'Unpublished ',
 'Science 291 (5507), 1304-1351 (2001) ',
 'Submitted (25-NOV-2003) Celera, 45 W Gude Drive, Rockville, MD 20850, USA ',
 'Science 291 (5507), 1304-1351 (2001) ',
 'Submitted (25-NOV-2003) Celera, 45 W Gude Drive, Rockville, MD 20850, USA ',
 'Science 301 (5641), 1898-1903 (2003) ',
 'Submitted (09-MAY-2003) Department of Eukaryotic Genomics, TIGR, The Institute for Genomic Research, 9712 Medical Center Drive, Rockville, MD 20850, USA ']

AUTHORS

In [178]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and Maier,U.G. ',
 'McFadden,G.I. ',
 'Lang,G., Wotton,D., Owen,M.J., Sewell,W.A., Brown,M.H., Mason,D.Y., Crumpton,M.J. and Kioussis,D. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Satou,Y., Shin-i,T., Kohara,Y. and Satoh,N. ',
 'Buell,C.R., Hart,A., Zismann,V., Karamycheva,S.A., Day,B., Staskawicz,B., Jin,H. and Baker,B. ',
 'Zhao,S., Adams,M.D., Nierman,W., Malek,J., de Jong,P. and Venter,J.C. ',
 'Venter,J.C., Adams,M.D., Myers,E.W., Li,P.W., Mural,R.J., Sutton,G.G., Smith,H.O., Yandell,M., Evans,C.A., Holt,R.A., Gocayne,J.D., Amanatides,P., Ballew,R.M., Huson,D.H., Wortman,J.R., Zhang,Q., Kodira,C.D., Zheng,X.H., Chen,L., Skupski,M., Subramanian,G., Thomas,P.D., Zhang,J., Gabor Miklos,G.L., Nelson,C., Broder,S., Clark,A.G., Nadeau,J., McKusick,V.A., Zinder,N., Levine,A.J., Roberts,R.J., Simon,M., Slayman,C., Hunkapiller,M., Bolanos,R., Delcher,A., Dew

TITLE

In [179]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Evidence that an amoeba acquired a chloroplast by retaining part of an engulfed eukaryotic alga ',
 'Direct Submission ',
 'The structure of the human CD2 gene and its expression in transgenic mice ',
 'Microbial populations from the subantarctic marine ecosystem ',
 'Direct Submission ',
 'Expressed genes in Ciona intestinalis (2002c) ',
 'Generation of EST sequences from Nicotiana benthamiana ',
 'Use of BAC End Sequences from Library RPCI-11 for Sequence-Ready Map Building ',
 'The sequence of the human genome ',
 'Direct Submission ',
 'The sequence of the human genome ',
 'Direct Submission ']

PUBMED

In [197]:
pub_list = []
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', locus, re.DOTALL)
# print(existe)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )
        pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
print(pub_list)

#procurar os ficheiros que tem o campo PUBMED, se não tiver vai valor vazio na lista (para fazer a relação genbank-PM)
file_locus_pub_list = []
for x in locus_list:
    aux_pub_list = []
    existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', x, re.DOTALL)
    # print(existe)
    if existe:
        for pubm in existe:
            m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )

            aux_pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
    else:
        aux_pub_list.append('')
    file_locus_pub_list.append(aux_pub_list)

print(file_locus_pub_list)

['191806', '1466314', '1496030', '16141072 ', '1614107', '16645617 ', '747619']
['747619']
[[''], ['191806'], ['1466314', '1496030', '16141072 ', '1614107', '16645617 '], ['747619']]


In [198]:
import requests
from bs4 import BeautifulSoup

def url_get_id_pm(item):
    url_id_pm = []
    for id_pm in item:
        url = f"https://pubmed.ncbi.nlm.nih.gov/{id_pm}/"
        url_id_pm.append(url)
    return url_id_pm

pm = ""
soup = []
content_pm = []
urls = url_get_id_pm(pub_list)
print(urls)

for url in urls:
	r2 = requests.get(url)
	pm += r2.content.decode('utf-8')
	soup.append(BeautifulSoup(r2.content, 'html.parser'))
	content_pm.append(r2.content.decode('utf-8'))

#print(soup)
#print(content_pm)




['https://pubmed.ncbi.nlm.nih.gov/191806/', 'https://pubmed.ncbi.nlm.nih.gov/1466314/', 'https://pubmed.ncbi.nlm.nih.gov/1496030/', 'https://pubmed.ncbi.nlm.nih.gov/16141072 /', 'https://pubmed.ncbi.nlm.nih.gov/1614107/', 'https://pubmed.ncbi.nlm.nih.gov/16645617 /', 'https://pubmed.ncbi.nlm.nih.gov/747619/']


AFFILIATION

In [151]:
import requests
r = requests.get('https://pubmed.ncbi.nlm.nih.gov/2549509')
pub = r.content.decode('utf-8')
print(pub)

<!DOCTYPE html>




<html lang="en" >
<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <!-- Mobile properties -->
    <meta name="HandheldFriendly" content="True">
    <meta name="MobileOptimized" content="320">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

  
  
  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.google-analytics.com">

  
  
    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.e3e95ed4a5de.css" type="text/css">
  

  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.452c70ce66f7.css" type="text/css">

  
    
  

  
    <link rel="stylesheet" href="https://cdn.n

In [183]:
import re
aff_list = []
for x in content_pm:
    existe = re.findall(r'"affiliation-\d"><sup class="key">\d</sup> \s*.*?(?=[.])', x, re.DOTALL)
#\s* zero ou mais espaços
#.* zero ou mais caracteres
#\. caractere ponto final -> procura affiliation até encontrar ponto final
    # print(existe)
    if existe:
        aux_list = []
        for c in existe:
        #m = re.findall(r'>\s*', c[0] )
        #o findall retorna uma lista, por isso temos que selecionar m[0] para fazer o replace
            m = c.replace('affiliation-1"><sup class="key">1</sup>','')
            aux_list.append(m)   #preenche para cada id a affiliation (mesmo que repetida)

    aux_list = list(dict.fromkeys(aux_list))  #remove os duplicados anteriormente inseridos
    aff_list.append(aux_list)

print(aff_list)
    # else:
        # print( "Padrão não encontrado" )


[['" Celera Genomics, 45 West Gude Drive, Rockville, MD 20850, USA'], ['" Celera Genomics, 45 West Gude Drive, Rockville, MD 20850, USA']]


AUTHOR, JOURNAL, TITLE, AFFILIATION

In [199]:
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np

In [200]:
Ids=[]
title_list = []
abstract = []
author = []
affiliation = []
journal = []
database = 'PubMed'
Entrez.email= "pg49836@alunos.uminho.pt"
# idlist= pub_list
idlist = []
handle = Entrez.efetch(db=database, id=pub_list, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    title_list.append(info.get("TI", "-"))
    author.append(info.get("AU", "-"))
    journal.append(info.get("SO", "-"))
    affiliation.append(info.get("AD", "-") )
    abstract.append(info.get("AB", "-"))
    print()
print(title_list)
print(abstract)
print(author)
print(affiliation)
print(journal)









['[Stimulating effect of human chorionic gonadotropin on the activity of guanidinoacetate-N-methyltransferase of rat testes: role of cyclic AMP].', 'Therapeutic management of triglycerides: an international perspective.', '[The relation between venous stasis and the occurrence of pain].', 'The transcriptional landscape of the mammalian genome.', '[The fate of corneal transplants from the Zurich eye bank].', 'Genome-wide analysis of mammalian promoter architecture and evolution.', 'The pressure-resistance relationships of regional resistances within the coronary circulation and the steal phenomenon.']
["A study was made of the influence of human chorionic gonadotropin (HCGT) on the activity of guanidineacetate-N-methyltranspherase in human testes. Guanidineacetate-methyltranspherase proved to be stimulated in the testes of the sexually immature rats in vivo under the action of HCGT on the 19th day after birth and from the 24th to the 27th day after birth. The activity of the enzy

CONNECTION TO DATABASE

In [201]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `LOCUS` (
        `Id_locus` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Accession` VARCHAR(100) NOT NULL,
        `Sequence_length` INT NULL,
        `Molecule_type` VARCHAR(20) NULL,
        `GB_division` VARCHAR(10) NULL,
        `Source` VARCHAR(200) NULL,
        `Organism` VARCHAR(200) NULL,
        `Locus_name` VARCHAR(45) NULL
        );
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES"
    loc_name_list = list(dict.fromkeys(loc_name_list))  #retirar duplicados (falta o último?)
    for index in range(len(loc_name_list)):
        if index == len(loc_name_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES ("X57223", "2098", "DNA linear", "INV", "Meloidogyne arenaria ", "Meloidogyne arenaria", "X57223 "), ("X53592", "769", "mRNA linear", "MAM", "Canis lupus familiaris (dog) ", "Canis lupus familiaris", "X53592 "), ("AAAEJ0019801", "21", "RNA linear", "ROD", "Mus musculus (house mouse) ", "Mus musculus", "AAAEJ0019801 "), ("Z33199", "760", "DNA linear", "BCT", "Mycoplasma capricolum subsp. capricolum ATCC 27343 ", "Mycoplasma capricolum subsp. capricolum ATCC 27343", "Z33199 ")
4 record inserted.


In [202]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `GENBANK` (
        `Id_GB` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `GB_id` VARCHAR(45) NOT NULL,
        `Definition` VARCHAR(400) NULL,
        `Modification_date` VARCHAR(11) NULL,
        `Version` VARCHAR(20) NOT NULL UNIQUE,
        `LOCUS_Id_locus` INT NOT NULL,
        `ORIGIN_Id_origin` INT NULL
    )
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES"
    

    for index in range(len(version_list)):
        sql_insert_origin = "INSERT INTO ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES" 
        sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
        Cursor.execute(sql_insert_origin)
        # print(Cursor.lastrowid)  #buscar o id do origin inserido
        id_inserted_origin = Cursor.lastrowid


        select_locus = f"SELECT Id_locus from LOCUS where Locus_name = '{loc_name_list[index]}'"
        Cursor.execute(select_locus)
        myresult = Cursor.fetchall()
        for res, in myresult:
            id_locus = res
        # print(id_locus)
        
        if index == len(version_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

4 record inserted.


In [203]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `PUBMED` (
        `Id_PM` INT NOT NULL AUTO_INCREMENT,
        `PM_ID` INT NOT NULL,
        `Journal` VARCHAR(100) NULL,
        `Title` VARCHAR(500) NULL,
        `Abstract` LONGTEXT NULL,
        PRIMARY KEY (`Id_PM`)
    )
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES"
    
    print(journal)
    print(len(title_list))
    print(len(abstract))
    print(len(pub_list))
    pub_list = list(dict.fromkeys(pub_list))  #tirar os duplicados (também n apanha o ultimo)
    for index in range(len(pub_list)):
        if index == len(pub_list)-1:
            sql += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\")"
        else:
            sql += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

['Probl Endokrinol (Mosk). 1977 Jan-Feb;23(1):79-83.', 'Am J Cardiol. 1992 Dec 14;70(19):26H-31H. doi: 10.1016/0002-9149(92)91087-k.', 'Phlebologie. 1992 Jan-Mar;45(1):33-7; discussion 38-9.', 'Science. 2005 Sep 2;309(5740):1559-63. doi: 10.1126/science.1112014.', 'Klin Monbl Augenheilkd. 1992 May;200(5):388-91. doi: 10.1055/s-2008-1045772.', 'Nat Genet. 2006 Jun;38(6):626-35. doi: 10.1038/ng1789. Epub 2006 Apr 28.', 'Basic Res Cardiol. 1978 Nov-Dec;73(6):607-17. doi: 10.1007/BF01906798.']
7
7
7
7 record inserted.


In [219]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_ids in range(len(file_locus_pub_list)):
        '''for index in range(len(author[index_ids])):
            #verificar se existe autor (evitar duplicados)
            select_ids_auth = f"SELECT Id_Author FROM AUTHORS WHERE Name = \"{author[index_ids]}\""
            Cursor.execute(select_ids_auth)
            myresult_auth = Cursor.fetchall()
            id_auth = None
            for res, in myresult_auth:
                id_affiliation = res'''

        if file_locus_pub_list[index_ids][0] != '':
            db_id_pm = []
            for pm_ids in file_locus_pub_list[index_ids]:
                select_pm_gb = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{pm_ids}\""
                
                #print(select_pm_gb)
                Cursor.execute(select_pm_gb)
                myresult_pm = Cursor.fetchall()
                id_pm = None
                for res, in myresult_pm:
                    id_pm = res
                db_id_pm.append(id_pm)

            print(db_id_pm)
            select_gb_pm = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_ids]}\""
            Cursor.execute(select_gb_pm)
            myresult_gb = Cursor.fetchall()
            id_gb = None
            for res, in myresult_gb:
                id_gb = res

            index = 0
            sql_pm_gb_relation = "INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES" 
            for id_pm in db_id_pm:
                if id_gb is not None and id_pm is not None:      
                    #verificar se existe relação entre ids
                    select_rel_gb_pm = f"SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = \"{id_pm}\" and GENBANK_Id_GB = \"{id_gb}\""
                    print(select_rel_gb_pm)
                    Cursor.execute(select_rel_gb_pm)
                    myresult_rel_pm_gb = Cursor.fetchall()
                    id_rel_pm_gb = None
                    for res_PUBMED_Id_PM,res_GENBANK_Id_GB, in myresult_rel_pm_gb:   #apenas selecionamos um dos id pq se não houver relação serão ambos null
                        id_rel_pm_gb = res_PUBMED_Id_PM     

                    #se null:
                    if id_rel_pm_gb is None:
                        if index == len(db_id_pm)-1:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb})"
                        else:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb}),"
                index += 1

            print(sql_pm_gb_relation)
            if sql_pm_gb_relation != "INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES":
                #remove ultimo caracter se este for uma ","
                if sql_pm_gb_relation.endswith(","):
                    sql_pm_gb_relation = sql_pm_gb_relation[:-1] #remove utimo caracter da string

                print(sql_pm_gb_relation)
                Cursor.execute(sql_pm_gb_relation)
                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

4
4
[11]
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "11" and GENBANK_Id_GB = "32"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES
[12, 13, 14, 15, 16]
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "12" and GENBANK_Id_GB = "33"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "13" and GENBANK_Id_GB = "33"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "14" and GENBANK_Id_GB = "33"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "15" and GENBANK_Id_GB = "33"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "16" and GENBANK_Id_GB = "33"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES
[17]
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "17" and GENBANK_Id_GB = "34"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB

In [288]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)

    for index_pubmed in range(len(pub_list)):
        
        affiliation_ids = []
        for index in range(len(affiliation[index_pubmed])):

            #verificar se existe afiliação (evitar duplicados)
            select_affiliation = f"SELECT Id_affiliation FROM AFFILIATION WHERE Description = \"{affiliation[index_pubmed][index]}\""
            Cursor.execute(select_affiliation)
            myresult_af = Cursor.fetchall()
            id_affiliation = None
            for res, in myresult_af:
                id_affiliation = res


            if id_affiliation is not None:
                affiliation_ids.append(id_affiliation)
            else:
                sql_insert_affiliation = "INSERT INTO AFFILIATION (Description) VALUES" 
                sql_insert_affiliation += f" (\"{affiliation[index_pubmed][index]}\")"
                print(sql_insert_affiliation)
                Cursor.execute(sql_insert_affiliation)
                # print(Cursor.lastrowid)  #buscar o id do affiliation inserido
                affiliation_ids.append(Cursor.lastrowid)

        sql = "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES"
        for index in range(len(author[index_pubmed])):
            if index > len(affiliation_ids)-1:
                if index == len(author[index_pubmed])-1:
                    sql += f" (\"{author[index_pubmed][index]}\", null)"
                else:
                    sql += f" (\"{author[index_pubmed][index]}\", null),"
            else:
                if index == len(author[index_pubmed])-1:
                    sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]})"
                else:
                    sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]}),"
        print(sql)
        Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Kozai K", 13), ("Moreno-Irusta A", 13), ("Iqbal K", 13), ("Winchester ML", 14), ("Scott RL", 13), ("Simon ME", 13), ("Muto M", 13), ("Parrish MR", 14), ("Soares MJ", 13)
INSERT INTO AFFILIATION (Description) VALUES ("Department of Basic and Translational Sciences, School of Dental Medicine, University of Pennsylvania, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Perelman School of Medicine, University of Pennsylvania, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Fox Chase Chemical Diversity Center, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Conifer Point Pharmaceuticals, Doylestown, PA, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("TR Bailey Consulting LLC, Phoenixville, PA, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Department of Basic and Translational Sciences, School of Dental Medicine, University of Pennsylvania, USA; Abramson Cancer Center, School of Medicine, Universi

IndexError: list index out of range